# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-req-build-ypi035_2
  Running command git clone -q git://github.com/Theano/Theano.git /tmp/pip-req-build-ypi035_2
  Created wheel for Theano: filename=Theano-1.0.4+21.g8f510a1f0-cp37-none-any.whl size=2667497 sha256=957c992afa1a40ec2cba32c731519a7b6ce6f8d43e7185b515b1ebdc74ba9cba
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nm9mxr3/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+21.g8f510a1f0
    Uninstalling Theano-1.0.4+21.g8f510a1f0:
      Successfully uninstalled Theano-1.0.4+21.g8f510a1f0
Requirement already up-to-date: tensorflow in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: keras in /srv/conda/envs/notebook/lib/python3.7/site-packages (2.3.1)
Requirement already up-to-date: pip in /srv/conda/envs/notebook/lib/python3.7/site-packages (19.3.1)
Requirement already up-to-date: pandas i

Requirement already up-to-date: numpy in /srv/conda/envs/notebook/lib/python3.7/site-packages (1.17.4)
Requirement already up-to-date: matplotlib in /srv/conda/envs/notebook/lib/python3.7/site-packages (3.1.1)
Requirement already up-to-date: sklearn in /srv/conda/envs/notebook/lib/python3.7/site-packages (0.0)


# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softmax', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [9]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 40)

Epoch 1/40
8000/8000 [==============================] - 2s 286us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 2/40
8000/8000 [==============================] - 2s 272us/step - loss: 0.4545 - accuracy: 0.7960
Epoch 3/40
8000/8000 [==============================] - 2s 278us/step - loss: 0.4302 - accuracy: 0.7983
Epoch 4/40
8000/8000 [==============================] - 3s 315us/step - loss: 0.4243 - accuracy: 0.8131
Epoch 5/40
8000/8000 [==============================] - 3s 313us/step - loss: 0.4181 - accuracy: 0.8210
Epoch 6/40
8000/8000 [==============================] - 2s 298us/step - loss: 0.4113 - accuracy: 0.8249
Epoch 7/40
8000/8000 [==============================] - 3s 323us/step - loss: 0.4040 - accuracy: 0.8307
Epoch 8/40
8000/8000 [==============================] - 3s 325us/step - loss: 0.3940 - accuracy: 0.8328
Epoch 9/40
8000/8000 [==============================] - 3s 326us/step - loss: 0.3841 - accuracy: 0.8422
Epoch 10/40
8000/8000 [==============================] - 3s 313u

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1507   88]
 [ 188  217]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [12]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.5036 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 220us/step - loss: 0.4814 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.4543 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 219us/step - loss: 0.4374 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 221us/step - loss: 0.4325 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 2s 188us/step - loss: 0.4306 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 166us/step - loss: 0.4297 - accuracy: 0.7959
Epoch 8/100
8000/8000 [==============================] - 1s 171us/step - loss: 0.4290 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 175us/step - loss: 0.4290 - accuracy: 0.7946
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 2s 236us/step - loss: 0.3570 - accuracy: 0.8555
Epoch 79/100
8000/8000 [==============================] - 2s 218us/step - loss: 0.3563 - accuracy: 0.8535
Epoch 80/100
8000/8000 [==============================] - 2s 237us/step - loss: 0.3517 - accuracy: 0.8553
Epoch 81/100
8000/8000 [==============================] - 2s 258us/step - loss: 0.3544 - accuracy: 0.8541
Epoch 82/100
8000/8000 [==============================] - 2s 268us/step - loss: 0.3500 - accuracy: 0.8571
Epoch 83/100
8000/8000 [==============================] - 2s 283us/step - loss: 0.3521 - accuracy: 0.8564
Epoch 84/100
8000/8000 [==============================] - 2s 230us/step - loss: 0.3525 - accuracy: 0.8562
Epoch 85/100
8000/8000 [==============================] - 2s 218us/step - loss: 0.3514 - accuracy: 0.8575
Epoch 86/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.3542 - accuracy: 0.8566
Epoch 87/100
8000/8000 [==============================] - 2

In [13]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1516   79]
 [ 202  203]]


# Initialising the 3rd ANN

In [15]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 283us/step - loss: 0.5281 - accuracy: 0.7955
Epoch 2/100
8000/8000 [==============================] - 2s 264us/step - loss: 0.4506 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 273us/step - loss: 0.4373 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 274us/step - loss: 0.4311 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 284us/step - loss: 0.4287 - accuracy: 0.7986
Epoch 6/100
8000/8000 [==============================] - 2s 264us/step - loss: 0.4279 - accuracy: 0.8004
Epoch 7/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.4273 - accuracy: 0.8006
Epoch 8/100
8000/8000 [==============================] - 2s 266us/step - loss: 0.4275 - accuracy: 0.8001
Epoch 9/100
8000/8000 [==============================] - 2s 274us/step - loss: 0.4272 - accuracy: 0.8024
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 2s 251us/step - loss: 0.3562 - accuracy: 0.8515
Epoch 79/100
8000/8000 [==============================] - 2s 260us/step - loss: 0.3545 - accuracy: 0.8519
Epoch 80/100
8000/8000 [==============================] - 2s 274us/step - loss: 0.3535 - accuracy: 0.8535
Epoch 81/100
8000/8000 [==============================] - 2s 277us/step - loss: 0.3520 - accuracy: 0.8528
Epoch 82/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.3507 - accuracy: 0.8544
Epoch 83/100
8000/8000 [==============================] - 2s 288us/step - loss: 0.3496 - accuracy: 0.8535
Epoch 84/100
8000/8000 [==============================] - 2s 282us/step - loss: 0.3484 - accuracy: 0.8545
Epoch 85/100
8000/8000 [==============================] - 2s 301us/step - loss: 0.3473 - accuracy: 0.8566
Epoch 86/100
8000/8000 [==============================] - 2s 249us/step - loss: 0.3465 - accuracy: 0.8576
Epoch 87/100
8000/8000 [==============================] - 2

In [16]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1532   63]
 [ 213  192]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

Epoch 1/10
7200/7200 [==============================] - 2s 252us/step - loss: 0.4927 - accuracy: 0.7969
Epoch 2/10
7200/7200 [==============================] - 2s 284us/step - loss: 0.4131 - accuracy: 0.8076
Epoch 3/10
7200/7200 [==============================] - 2s 318us/step - loss: 0.3871 - accuracy: 0.8238
Epoch 4/10
7200/7200 [==============================] - 2s 289us/step - loss: 0.3671 - accuracy: 0.8487
Epoch 5/10
7200/7200 [==============================] - 2s 260us/step - loss: 0.3566 - accuracy: 0.8557
Epoch 6/10
7200/7200 [==============================] - 2s 269us/step - loss: 0.3516 - accuracy: 0.8583
Epoch 7/10
7200/7200 [==============================] - 2s 277us/step - loss: 0.3493 - accuracy: 0.8587
Epoch 8/10
7200/7200 [==============================] - 2s 279us/step - loss: 0.3474 - accuracy: 0.8599
Epoch 9/10
7200/7200 [==============================] - 2s 270us/step - loss: 0.3455 - accuracy: 0.8596
Epoch 10/10
800/800 [==============================] - 0s 117us/

7200/7200 [==============================] - 2s 240us/step - loss: 0.4130 - accuracy: 0.8281
Epoch 6/10
7200/7200 [==============================] - 2s 270us/step - loss: 0.4100 - accuracy: 0.8321
Epoch 7/10
7200/7200 [==============================] - 2s 268us/step - loss: 0.4073 - accuracy: 0.8318
Epoch 8/10
7200/7200 [==============================] - 2s 277us/step - loss: 0.4035 - accuracy: 0.8328
Epoch 9/10
7200/7200 [==============================] - 2s 300us/step - loss: 0.3963 - accuracy: 0.8361
Epoch 10/10
800/800 [==============================] - 0s 74us/step
Epoch 1/10
7200/7200 [==============================] - 2s 281us/step - loss: 0.4875 - accuracy: 0.7954
Epoch 2/10
7200/7200 [==============================] - 2s 289us/step - loss: 0.4265 - accuracy: 0.7957
Epoch 3/10
7200/7200 [==============================] - 2s 303us/step - loss: 0.4209 - accuracy: 0.7957
Epoch 4/10
7200/7200 [==============================] - 2s 293us/step - loss: 0.4170 - accuracy: 0.8206
Epoch 5

# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [19]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

Epoch 1/10
7200/7200 [==============================] - 3s 358us/step - loss: 0.4992 - accuracy: 0.7961
Epoch 2/10
7200/7200 [==============================] - 2s 307us/step - loss: 0.4389 - accuracy: 0.7971
Epoch 3/10
7200/7200 [==============================] - 2s 325us/step - loss: 0.4326 - accuracy: 0.7971
Epoch 4/10
7200/7200 [==============================] - 2s 305us/step - loss: 0.4296 - accuracy: 0.7971
Epoch 5/10
7200/7200 [==============================] - 2s 329us/step - loss: 0.4273 - accuracy: 0.7971
Epoch 6/10
7200/7200 [==============================] - 2s 320us/step - loss: 0.4288 - accuracy: 0.7971
Epoch 7/10
7200/7200 [==============================] - 2s 317us/step - loss: 0.4284 - accuracy: 0.8085
Epoch 8/10
7200/7200 [==============================] - 2s 333us/step - loss: 0.4269 - accuracy: 0.8218
Epoch 9/10
7200/7200 [==============================] - 3s 364us/step - loss: 0.4281 - accuracy: 0.8211
Epoch 10/10
800/800 [==============================] - 0s 141us/

7200/7200 [==============================] - 2s 295us/step - loss: 0.4119 - accuracy: 0.8222
Epoch 6/10
7200/7200 [==============================] - 2s 304us/step - loss: 0.4064 - accuracy: 0.8217
Epoch 7/10
7200/7200 [==============================] - 2s 317us/step - loss: 0.4011 - accuracy: 0.8239
Epoch 8/10
7200/7200 [==============================] - 2s 307us/step - loss: 0.4049 - accuracy: 0.8210
Epoch 9/10
7200/7200 [==============================] - 2s 347us/step - loss: 0.4044 - accuracy: 0.8224
Epoch 10/10
800/800 [==============================] - 0s 110us/step
Epoch 1/10
7200/7200 [==============================] - 2s 323us/step - loss: 0.5060 - accuracy: 0.7956
Epoch 2/10
7200/7200 [==============================] - 2s 306us/step - loss: 0.4392 - accuracy: 0.7957
Epoch 3/10
7200/7200 [==============================] - 2s 317us/step - loss: 0.4333 - accuracy: 0.7957
Epoch 4/10
7200/7200 [==============================] - 2s 317us/step - loss: 0.4300 - accuracy: 0.7957
Epoch 

# Tuning the ANN

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [21]:
#parameters = {'batch_size': [10, 24, 32],
#              'epochs': [50, 100, 200],
#              'optimizer': ['adam', 'rmsprop']}

#grid_search = GridSearchCV(estimator = classifier,
#                           scoring = 'accuracy',
#                           param_grid = parameters,
#                           cv = 10)

#grid_search = grid_search.fit(X_train, y_train)

#best_parameters = grid_search.best_params_
#best_accuracy = grid_search.best_score_

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)